In [181]:
import numpy as np
import math
import pprint as pp
from scipy import *
from scipy.sparse import *

## Generation de graphes

In [182]:
handmadeGraph1 = {1: {2:0.5,3:0.5},
                 2: {4:0.6},
                 3: {4:0.4,5:0.6},
                 4: {},
                 5: {},
                }

handmadeGraph=dok_matrix((7,7),dtype=float32)
handmadeGraph[0,1] =0.5
handmadeGraph[0,2] =0.5
handmadeGraph[1,3] =0.6
handmadeGraph[2,3] =0.4
handmadeGraph[2,4] =0.6

print("Handmade Graph")
print(handmadeGraph)


nodes = np.arange(handmadeGraph.shape[0])

def randomGraph(nodes):
    m=max(nodes)+1
    graph = dok_matrix((m,m),dtype=float32)
    for n1 in nodes:
        for n2 in nodes:
            if(n1!=n2):
                graph[n1,n2]=np.random.rand()
    return graph
print("Random Graph")
print(randomGraph(nodes))

Handmade Graph
  (0, 1)	0.5
  (0, 2)	0.5
  (1, 3)	0.6
  (2, 3)	0.4
  (2, 4)	0.6
Random Graph
  (0, 1)	0.03156902
  (0, 2)	0.49405158
  (0, 3)	0.6980423
  (0, 4)	0.33177012
  (0, 5)	0.026152728
  (0, 6)	0.8809852
  (1, 0)	0.5689927
  (1, 2)	0.49533373
  (1, 3)	0.4199267
  (1, 4)	0.54160166
  (1, 5)	0.5550246
  (1, 6)	0.15587775
  (2, 0)	0.3220729
  (2, 1)	0.66631955
  (2, 3)	0.32078075
  (2, 4)	0.22548972
  (2, 5)	0.26092488
  (2, 6)	0.6807767
  (3, 0)	0.4706855
  (3, 1)	0.51292473
  (3, 2)	0.39657506
  (3, 4)	0.9988955
  (3, 5)	0.8106461
  (3, 6)	0.73198074
  (4, 0)	0.76005745
  (4, 1)	0.48004043
  (4, 2)	0.86843073
  (4, 3)	0.0057362984
  (4, 5)	0.42348614
  (4, 6)	0.98618335
  (5, 0)	0.073831804
  (5, 1)	0.0012110415
  (5, 2)	0.26615423
  (5, 3)	0.40700644
  (5, 4)	0.41676894
  (5, 6)	0.62926596
  (6, 0)	0.03957735
  (6, 1)	0.45346314
  (6, 2)	0.4625029
  (6, 3)	0.9781842
  (6, 4)	0.29001147
  (6, 5)	0.59078664


## Inference 
Generation de cascades

In [183]:

def genCascade(graph,startNode,startTime=0):
    ''' Receive graph and starting infected node generate a cascade 
        Args
            graph(node to child representation)
            cascade {node : time} 
    '''
    cascade = {startNode : startTime}
    lastInfected = [startNode]
    infected_next = {}
    time = startTime+1
    while len(lastInfected)> 0:
        for infected in lastInfected:
            for (_,child),pct in graph[infected,:].items():
                if (child not in cascade) and (child not in infected_next):
                    if (np.random.rand()<pct):
                        infected_next[child] = time
        cascade.update(infected_next)
        lastInfected = list(infected_next.keys())
        infected_next = {}
        time +=1
        
    return cascade

In [184]:
nbCascades = 200
cascades = [genCascade(handmadeGraph,0) for i in range(nbCascades)]
pp.pprint(cascades[:3])

[{0: 0, 1: 1, 3: 2}, {0: 0, 1: 1, 2: 1, 4: 2}, {0: 0, 2: 1, 4: 2}]


## Independant cascades apprentissage


### 2008 Saito
Les temps d'infections contiguë <br>
$D(i)$  all newly infected at time $i$  <br>
$D = D(0) \cup D(1) \cup ... \cup D(n)$
#### Maximimum likelyhood expectation
Etapes : 
1. On donne une première estimation des arêtes du graph : $k_{u,v}$ <br>
- On calcule $P_{w}$ en fonction de $k_{u,v}$
- On calcule par ML $k_{u,v}$ en fonction de $P_{w}$
- Retour à l'étape 2 jusqu'à convergence

#### likelyhood
On peut vérifier que l'algorithme augmente sa vraisemblance à chaque étapes : <br>
On définit la vraisemblance qu'un noeud soit infecté ou non pour un épisode de diffusion Ds donné :<br>
$ P(v_t=t | D_s) = 1-\prod_{u \in Ds \land\\ v_{t} = u_{t+1}}  1-\theta_{u,v} $<br> 
$ P(v_t=\infty | D_s) = \prod_{u \in Ds }  1-\theta_{u,v} $<br>
La vraisemblance d'une cascades c'est la probabilité que pour tous les noeuds du graphes : 
- Si le noeud fait partie de la cascade : qu'il ait été infecté par un de ses prédécesseur
- Si le noeud ne fait pas partie de la cascade : qu'il n'ai jamais été infectés par les noeuds de la cascade
<br>

<br> Puis on calcule la vraisemblance d'un Episode Ds : 

$L(Ds; \theta) =\left[ \prod_{v \notin Ds} P(v_t = \infty | D_s) \right]* 
\left[\prod_{v \in Ds} P(v_t = t | D_s)  \right]$ <br>

$L(Ds; \theta) = \left[ \prod_{v \notin Ds}1-\prod_{u \in Ds \land\\ v_{t} = u_{t+1}}  1-\theta_{u,v} \right]* 
\left[\prod_{v \in Ds} \prod_{u \in Ds }  1-\theta_{u,v} \right] $ <br>

Expression de la log-likelyhood : <br>
$L(Ds; \theta) =\left[ \sum_{v \notin Ds} log (P(v_t = \infty | D_s)) \right]+
\left[\sum_{v \in Ds} log (P(v_t = t | D_s))  \right]$ <br>


Donc pour l'ensemble des Epsiodes on a : <br>
$ L(D,\theta ) =\sum_{Ds \in D} L(Ds; \theta) $ <br>



In [185]:
def cascade_repr(cascade):
    ''' Transforme une cascade en list ayant pour indice 
        le temps d'infection t et pour contenu une list contenant les noeuds infectés au temps t'''
    
    maxT  = max(cascade.values())+1
    Ds = [[] for i in range(maxT)]
    for (n,t) in cascade.items():
        Ds[t].append(n)
    return Ds
D = [cascade_repr(c) for c in cascades]
print(cascades[:2],'\n',D[:2])

[{0: 0, 1: 1, 3: 2}, {0: 0, 1: 1, 2: 1, 4: 2}] 
 [[[0], [1], [3]], [[0], [1, 2], [4]]]


In [186]:
def nodes_in_Ds(Ds):
    uniques = []
    for nodes in Ds : 
        for n in nodes : 
            uniques.append(n)
    return uniques

In [187]:
def P_sw(g,Ds,w):
    '''Vraisemblanblance qu'un noeud w soit ou non infecté sachant la cascade Ds 
        et les paramètres du graphe '''
    t = None
    for i,nodes in enumerate(Ds):
        if (w in nodes):
            t = i
    if (t == 0): # si le noeud est le premier
        return 1
    if (t is None): # si le noeud n'est pas dans l'episode de diffusion
        return P_NotInfected_sw(g,Ds,w)
    else :  # si le noeud est dans l'épisode de diffusion
        return P_Infected_sw(g,Ds,w,t)

def P_Infected_sw(g,Ds,w,t):
    ''' Vraisemblance de l'infection positive d un noeud à l'étape t sachant le graphe '''
    return 1 - np.prod ([1-g[parent,w] for parent in Ds[t-1]])


def P_NotInfected_sw(g,Ds,w):
    ''' Vraisemblance Qu'un noeud ne soit pas infecté dans un epsiode Ds sachant le 
        graphe'''
    return np.prod([1-g[parent,w] for parent in nodes_in_Ds(Ds)])

In [188]:
def Expectation(g,D):
    ''' Calcule l'ensemble des P_ws 
        P_ws[idD][node]->proba'''
    p_sw = [{n:P_sw(g,Ds,n) for n in nodes_in_Ds(Ds)} for Ds in D]
    return p_sw

In [189]:
def D_plus_uv_id(D,u,v):
    ''' pour chaque couples u,v renvoit l'ensemble des 
    Ds (episode infections) ou u precede v'''
    D_plus = []
    for i,Ds in enumerate(D): 
        for t in range(1,len(Ds)):
            if (u in Ds[t-1] and v in Ds[t]):
                D_plus.append(i)
                break
    return D_plus

def D_minus_uv_id(D,u,v):
    '''Pour chaque couple u,v renvoit l'ensemble des 
    Ds(episode infection) ou u est present mais v ne le suit pas'''
    D_minus_id=[]
    for s,Ds in enumerate(D) : 
        for t in range(1,len(Ds)):
            if (u in Ds[t-1] and v not in Ds[t]):
                D_minus_id.append(s)
                break
        if (u in Ds[-1]):
            D_minus_id.append(s)
    return D_minus_id

def D_minus_uv_len(D,u,v):
    '''Pour chaque couple u,v renvoit le cardinal de l'ensemble des 
    Ds(episode infection) ou u est present mais v ne le suit pas'''
    D_minus_len=0
    for Ds in D : 
        for t in range(1,len(Ds)):
            if (u in Ds[t-1] and v not in Ds[t]):
                D_minus_len+=1
                break
        if (u in Ds[-1]):
            D_minus_len+=1
    return D_minus_len

In [190]:
def llikelyhood_Ds(g,Ds):
    '''Calcule la log vraisemblance d'une cascade selon le graph '''
    ll = 0
    for v,u in g.keys() :
        ll += np.log(P_sw(g,Ds,u))
    return ll

def llikelyhood(g,D):
    total = 0
    for Ds in D : 
        total +=llikelyhood_Ds(g,Ds)
    return total
    return sum([llikelyhood_Ds(g,Ds) for Ds in D])

In [191]:
def Maximisation_uv(g,D_plus_id,Dminus_len,p_sw,u,v):
    '''Calcule les nouveaux paramètre pour l'arete u,v '''
    if ((len(D_plus_id[u][v])+Dminus_len[u][v]) == 0):
        #raise Exception(f"{u}-{v} Division zero")
        return 0
    return (1/(len(D_plus_id[u][v])+Dminus_len[u][v])) *sum([g[u,v]/p_sw[i][v] for i in D_plus_id[u][v]])


def Maximisation(g,D_plus_id,Dminus_len,p_sw):
    ''' Calcule les nouveaux paramètres pour le graphe'''
    gprime = dok_matrix(g.shape,dtype=float32)
    for u,v in g.keys():
        if u != v:
            gprime[u,v] = Maximisation_uv(g,D_plus_id,Dminus_len,p_sw,u,v)
    return gprime


In [192]:
def remove_edges(g,nodes,D_plus):
    ''' Retire en place tous les arêtes (u,v) ou il n'existe pas de cascade ou 
        u précède v. (=>on ne peut rien dire sur les probabilité que u infecte v)
    '''
    for n1 in nodes : 
        for n2 in nodes : 
            if len(D_plus[n1][n2]) == 0:
                g[n1,n2] = 0

In [193]:
def MSE(g1,g2):
    '''calcule la mean square error entre 2 graphes '''
    assert g1.shape == g2.shape
    return np.sum((g1-g2).power(2))

In [194]:
def EM_IC(D,nodes,threshold = 10**(-7),debug = False):
        # initalisation
    g = randomGraph(nodes)    
    p_sw = None
    D_plus_id =   {v:{u:D_plus_uv_id(D,v,u) for u in nodes} for v in nodes}
    D_minus_len = {v:{u:D_minus_uv_len(D,v,u)for u in nodes} for v in nodes}
    remove_edges(g,nodes,D_plus_id)
    
    if debug:
        ll = -float("inf") # ll : loglikelyhood
    
    loop = True
    while loop:
        p_sw = Expectation(g,D)
        next_g = Maximisation(g,D_plus_id,D_minus_len,p_sw)
        
        loop = MSE(g,next_g) > threshold
        g = next_g
        
        if debug : 
            new_ll = llikelyhood(g,D)
            if new_ll < ll : 
                print(f"Likelyhood Error : descreasing : from {ll} to {new_ll}")
            ll = new_ll
            
    return g

nodes = set()
for c in cascades:
    nodes.update(c.keys())
print(nodes)

D = [cascade_repr(c) for c in cascades]
finalGraph = EM_IC(D,nodes,debug=True)
print(finalGraph)

{0, 1, 2, 3, 4}
  (0, 1)	0.59
  (0, 2)	0.515
  (1, 3)	0.5356452
  (1, 4)	0.00023212147
  (2, 3)	0.45048556
  (2, 4)	0.6212585


### Methode 2
Les temps d'infections non contiguë : 
Un noeud peut désorais être infecté non seulement par ceux qui ont été infecté<br>
On doit alors redéfinir la likelyhood : <br>
$ P(v_t=t | D_s) = 1-\prod_{u \in Ds \land\\ v_{t} < u_{t}}  1-\theta_{u,v} $<br> 
$ P(v_t=\infty | D_s) = \prod_{u \in Ds }  1-\theta_{u,v} $<br>
$L(Ds; \theta) =\left[ \prod_{v \notin Ds} P(v_t = \infty | D_s) \right]* 
\left[\prod_{v \in Ds} P(v_t = t | D_s)  \right]$ <br>


In [195]:
def P_sw_not_contiguous(g,Ds,w):
    ''' Vraisemblance de l infection d un noeud sachant le graphe 
    et les tous noeuds précedemment activés (tps infection non contigue) '''
    not_activated = 1
    t = None
    for i,nodes in enumerate(Ds):
        if (w in nodes):
            t = i
    if (t == 0): # si le noeud est le premier
        return 1
    if (t is None): # si le noeud n'est pas dans l'episode de diffusion
        return P_NotInfected_sw(g,Ds,w)
    else :  # si le noeud est dans l'épisode de diffusion        
        return P_sw_infected_not_contiguous(g,Ds,w,t)

def P_sw_infected_not_contiguous(g,Ds,w,t):
    preceding_nodes = [] # on regroupe les noeuds des épisodes précedents
    for nodes in Ds[:t]:
        for n in nodes : 
            preceding_nodes.append(n)
        
    return 1 - np.prod ([1-g[parent,w] for parent in preceding_nodes])

In [196]:
def llikelyhood_Ds_notContiguous(g,Ds):
    '''Calcule la log vraisemblance d'une cascade selon le graph '''
    ll = 0
    for v,u in g.keys() :
        ll += np.log(P_sw_not_contiguous(g,Ds,u))
    return ll

def llikelyhood_notContiguous(g,D):
    total = 0
    for Ds in D : 
        total +=llikelyhood_Ds(g,Ds)
    return total
    return sum([llikelyhood_Ds(g,Ds) for Ds in D])

In [197]:
def Expectation2(g,D):
    ''' Calcule l'ensemble des P_ws 
        P_ws[idD][node]->proba'''
    p_sw = [{n:P_sw_not_contiguous(g,Ds,n) for n in nodes_in_Ds(Ds)} for Ds in D]
    return p_sw

In [198]:
def D_plus_uv_id2(D,u,v):
    ''' Pour chaque couples u,v renvoit l'ensemble des 
        Ds (episode infections) ou t_u < t_v'''
    D_plus = []
    
    for i,Ds in enumerate(D) : 
        preceding_nodes = []
        for t in range(0,len(Ds)):
            if (v in Ds[t]):
                if (u in preceding_nodes):
                    D_plus.append(i)
                    break
            else : 
                preceding_nodes +=Ds[t]
    return D_plus

def D_minus_uv2_len(D,u,v):
    '''Pour chaque couple u,v renvoit l'ensemble des 
    Ds(episode infection) ou u est present et non(t_u < t_v)'''
    D_minus_len = 0
    for Ds in D : 
        u_in_Ds = False
        v_in_Ds = False
        for t in range(0,len(Ds)):
            if (u in Ds[t]):
                u_in_Ds = True
            if (u_in_Ds and v in Ds[t]):
                v_in_Ds = True
                break
        if (u_in_Ds and not v_in_Ds):
            D_minus_len+=1
    return D_minus_len

In [206]:
def EM_IC_2(D,nodes,threshold=10**(-7), debug=False):
    g = randomGraph(nodes)    
    p_sw = None
    D_plus_id =   {v:{u:D_plus_uv_id2(D,v,u) for u in nodes} for v in nodes}
    D_minus_len = {v:{u:D_minus_uv2_len(D,v,u)for u in nodes} for v in nodes}
    remove_edges(g,nodes,D_plus_id)

    if (debug):
        ll = -float('inf')
    
    loop = True
    while loop:
        p_sw = Expectation2(g,D)
        g_next = Maximisation(g,D_plus_id,D_minus_len,p_sw)
        
        loop = MSE(g,g_next)<threshold
        g = g_next
        
        if debug : 
            new_ll = llikelyhood_notContiguous(g,D)
            if new_ll < ll : 
                print(f"Likelyhood Error : descreasing : from {ll} to {new_ll}")
            ll = new_ll

    return g

D = [cascade_repr(c) for c in cascades]
finalGraph = EM_IC_2(D,nodes,debug=True)
print(finalGraph)

  (0, 1)	0.59
  (0, 2)	0.515
  (0, 3)	0.31027412
  (0, 4)	0.2654719
  (1, 3)	0.50248927
  (1, 4)	0.31386676
  (2, 3)	0.24735372
  (2, 4)	0.3194314


## Evaluation

On va évaluer les modèles par MeanAveragePrecision (MAP): <br>
On peut caculer $AP$ pour un épisode ($Ds$) en particulier : <br><br>
$ AP(Ds) =  \sum_{i=1}^{|U^{Ds}|} \frac{ |\{ U_1^{Ds},...,U_i^{Ds} \} \cap Ds |}{i}  dx$ <br>
$ AP(Ds) = \sum_{i=1}^{|U^{Ds}|} \frac{TruePositive}{TruePositive+FalseNegative}dx$ <br><br>
$ dx = recall(i) -recall(i-1) $<br>
<br>
$ MAP = \frac{1}{D}\sum_{Ds\in |D|} AP(Ds)$

On va d'abord calculer les probabilité d'infection d'un noeud connaissant la source 
par génération d'épisodes : 

In [207]:
def Pws_gs(graph,source,nbEpisode=100):
    ''' Calcule la probabilité qu'un noeud soit infecté connaissant 
        une source et un graph de diffusion
        On calcule par moyenne sur echantionnage d'episodes d'infections '''
    proba_infected = {n:0 for n in range(graph.shape[0])}
    for i in range(nbEpisode):
        c = genCascade(graph,source)
        for node in c : 
            proba_infected[node]+=1/nbEpisode
    return proba_infected

In [208]:
def AP(Ds,graph):
    '''Average Precision pour un episode Ds '''
    pws_gs =Pws_gs(graph,Ds[0][0])
    U_d = sorted(pws_gs,key=pws_gs.get,reverse=True) # sort par ordre decroissant
    DsNodeSet = set(nodes_in_Ds(Ds)) # noeuds faisant partie de l'episode d'infection Ds
    ap =0
    for i,node in enumerate(U_d) : 
        if (node in DsNodeSet):
            tp = len(DsNodeSet.intersection(U_d[:(i+1)])) # true positive
            ap+= tp/(i+1) # precision = tp/(tp+fp)
    ap/= len(DsNodeSet)
    return ap

def MAP(D,graph):
    return sum([AP(Ds,graph)for Ds in D])/len(D)    

In [209]:
print("original  graph MAP score : ",MAP(D,handmadeGraph))
print("inference graph MAP score : ",MAP(D,finalGraph))

original  graph MAP score :  0.9361875000000001
inference graph MAP score :  0.9292430555555556


# Test avec matrices Sparses : 

In [210]:
def GenerateData(nbNodes,nbCascades,density=0.1):
    ''' Genere un graph aléatoire et des données '''
    sparseGraph = dok_matrix(random(nbNodes,nbNodes,density=density))
    cascades = []
    for i in range(nbCascades):
        startNode = np.random.randint(nbNodes)
        cascades.append(genCascade(sparseGraph,startNode))
        # cascade representation : 
    D = [cascade_repr(c) for c in cascades]
    return sparseGraph,D

def ModelScore(D,trainD,testD):
    ''' Apprends un modele sur ensemble de trainD 
        et renvoi le résultat sur un ensemble de testD'''
    nodes = set()
    for Ds in D:
        for Ds_t in Ds : 
            nodes.update(set(Ds_t))
    predictedGraph = EM_IC_2(trainD,nodes)
    return MAP(testD,predictedGraph)

In [212]:
# Params : 
nbNodes = 200
nbCascades = 400
originalGraph,D = GenerateData(nbNodes,nbCascades,0.007)
trainD,testD = D[int(nbCascades/2):],D[:int(nbCascades/2)]
predictedScore = ModelScore(D,trainD,testD)
originalScore = MAP(D,originalGraph)
print("original  graph MAP score : ",originalScore)
print("predictedGraph MAP score : ",predictedScore)

original  graph MAP score :  0.9611435751823965
predictedGraph MAP score :  0.812665544932929
